# Machine Learning Challenge 2 : Youtube Comments Toxicity 



L’objectif de ce défi est de comprendre et prédire la notion de toxicité dans une vidéo
Youtube. Il s’agit de déterminer dans quelle mesure l’espace des commentaires des médias
français est touché par les débats brutaux et agressifs. Nous posons comme objectif de
comprendre où et quand l’agressivité en ligne est retrouvée et quels facteurs la limite et la
favorisent. D’abord, en abordant la question sous la forme d’un problème de régression
visant à prédire le nombre d’insultes présentes dans les commentaires associés à une
vidéo. Ensuite, en construisant puis prédisant un indice original de toxicité à partir des
variables présentes dans la base.

## Présentation et analyse des données 

### Importation des bibliothèques et chargement des données

In [21]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from random import shuffle
from sklearn import tree
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, accuracy_score, f1_score, precision_score, recall_score, mean_squared_error

# import des données

pd.set_option('display.max_columns', None)
df = pd.read_csv('challenge_youtube_toxic.csv', encoding = 'latin1', engine = 'python', sep=';')

In [22]:
df.head()

,video_id_court,video_id,channel_id,nbrMot,nbrMotInsulte,nbrMotAllong,nbrMotMAJ,nbrExclMark,nbrQuestMark,nbrMotMoyenne,nbrMotInsulteMoyenne,nbrMotAllongMoyenne,nbrMotMAJMoyenne,nbrExclMarkMoyenne,nbrQuestMarkMoyenne,thread_count,comment_count,message_count,discussion_count,distinct_authors_count,authors_3channels_count,liked_authors_count,channel_name,subscriberCount,viewCount,categorie_new,categ_inst,Unnamed: 27
0,Uvvsvw_wxOs,VVU5R0d6QWhodmhKTzFoTDEwLUJjZ05BLlV2dnN2d193eE9z,UC9GGzAhhvhJO1hL10-BcgNA,3813,18,4,145,79,60,"30,2619","0,1428571","0,03174603","1,150794","0,6269841","0,4761905",45,81,126,28,58,47,23,LeHuffPost,339922,225393312,Core,Pure Players,NaN
1,mCy1ZBdttwA,VVVYS0pyWWN6WTJfZkpFWmdGUEdZMEhRLm1DeTFaQmR0dHdB,UCXKJrYczY2_fJEZgFPGY0HQ,28,2,0,1,0,0,"9,333333","0,6666667",0,"0,3333333",0,0,3,0,3,0,3,1,0,CNEWS,189971,132125693,Core,TV,NaN
2,iU3MyHMK5nU,VVU5R0d6QWhodmhKTzFoTDEwLUJjZ05BLmlVM015SE1LNW5V,UC9GGzAhhvhJO1hL10-BcgNA,3804,68,9,56,75,54,"27,56522","0,4927536","0,06521739","0,4057971","0,5434783","0,3913043",52,86,138,22,83,49,36,LeHuffPost,339922,225393312,Core,Pure Players,NaN
3,ZUim3AeURtk,VVViWjBDc3I5b3c2WGprZGQ1NDJPbWh3LlpVaW0zQWVVUnRr,UCbZ0Csr9ow6Xjkdd542Omhw,4,0,1,2,0,0,2,0,"0,5",1,0,0,2,0,2,0,2,0,0,Telegramme,2862,2481100,Niche,Presse_regionale,NaN
4,n5Tg2vhLCAM,VVVoWldzNlBKWTBoTkQzODRkMl9ScmhRLm41VGcydmhMQ0FN,UChZWs6PJY0hND384d2_RrhQ,737,3,1,18,5,6,"32,04347826","0,130434783","0,043478261","0,782608696","0,217391304","0,260869565",17,6,23,4,18,7,7,Les Echos,30010,6710737,Core,Presse nationale,NaN


Il faut changer le type de certaines colonnes, et supprimer la colonne inutile : 

In [23]:
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]

def convert_into_float(string):
    if type(string)==float:
        return string
    return float (string.replace(',','.'))

df['nbrMotMoyenne'] = df['nbrMotMoyenne'].apply(convert_into_float)
df['nbrMotInsulteMoyenne'] = df['nbrMotInsulteMoyenne'].apply(convert_into_float)
df['nbrMotAllongMoyenne'] = df['nbrMotAllongMoyenne'].apply(convert_into_float)
df['nbrMotMAJMoyenne'] = df['nbrMotMAJMoyenne'].apply(convert_into_float)
df['nbrExclMarkMoyenne'] = df['nbrExclMarkMoyenne'].apply(convert_into_float)
df['nbrQuestMarkMoyenne'] = df['nbrQuestMarkMoyenne'].apply(convert_into_float)
len(df.dropna()) # permet de déterminer la qualité de saisie de la donnée. Ici il n'y a donc pas de données manquantes

46102

On va maintenant pouvoir calculer certaines statistiques sur les données afin de mieux les comprendre. 

In [24]:
df.describe()


,nbrMot,nbrMotInsulte,nbrMotAllong,nbrMotMAJ,nbrExclMark,nbrQuestMark,nbrMotMoyenne,nbrMotInsulteMoyenne,nbrMotAllongMoyenne,nbrMotMAJMoyenne,nbrExclMarkMoyenne,nbrQuestMarkMoyenne,thread_count,comment_count,message_count,discussion_count,distinct_authors_count,authors_3channels_count,liked_authors_count,subscriberCount,viewCount
count,46102.000000,46102.000000,46102.000000,46102.000000,46102.000000,46102.000000,46102.000000,46102.000000,46102.000000,46102.000000,46102.000000,46102.000000,46102.000000,46102.000000,46102.000000,46102.000000,46102.000000,46102.000000,46102.000000,46102.000000,4.610200e+04
mean,1435.853564,9.418225,3.974665,21.922823,21.823066,13.591276,25.264768,0.189175,0.080849,0.583485,0.633837,0.276134,22.219145,25.700816,47.919960,5.040085,28.726021,14.502646,10.974036,172108.752744,9.255061e+07
std,7670.789422,61.616732,21.367879,130.038591,120.607203,68.803038,31.979351,0.444356,0.305948,3.114312,1.600474,0.950349,117.441322,130.742237,234.791106,19.578795,140.334032,56.251117,50.607716,119508.318824,6.778302e+07
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,3.109100e+04
25%,18.000000,0.000000,0.000000,0.000000,0.000000,0.000000,9.769231,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000,1.000000,0.000000,87793.000000,4.635634e+07
50%,72.000000,0.000000,0.000000,0.000000,1.000000,1.000000,18.630032,0.000000,0.000000,0.000000,0.250000,0.040161,3.000000,0.000000,4.000000,0.000000,3.000000,2.000000,1.000000,180173.000000,6.388093e+07
75%,398.000000,3.000000,1.000000,6.000000,9.000000,4.000000,31.000000,0.222222,0.068821,0.400000,0.750000,0.333333,9.000000,5.000000,16.000000,2.000000,11.000000,7.000000,4.000000,217835.000000,1.321257e+08
max,662808.000000,7227.000000,1416.000000,14270.000000,11093.000000,5520.000000,1349.500000,14.000000,28.000000,162.000000,104.000000,120.750000,8734.000000,11195.000000,19336.000000,1235.000000,9837.000000,3762.000000,3732.000000,568615.000000,2.414350e+08


In [40]:
videos_par_chaines=[len(df[df['channel_name'] == name].video_id.unique()) for name in df.channel_name.unique()]
print(' Nombre de chaînes différentes = ', len(df.channel_id.unique()), ", Nombre moyen de vidéos par chaîne : ", round(len(df)/len(df.channel_id.unique())) )
print("Ecart type du nombre de vidéos par chaînes : ", round( df.groupby('channel_name').video_id.count().std()))

 Nombre de chaînes différentes =  58 , Nombre moyen de vidéos par chaîne :  795
Ecart type du nombre de vidéos par chaînes :  1613


On peut voir que le nombre de vidéos par chaîne est réparti sur une grande plage de données.
Trouvons maintenant la chaîne avec le plus d'abonnés. 

In [43]:
# on trouve la chaîne YouTube avec le plus d'abonnés.

channel_max = ''
max_val = 0
for name in df.channel_name.unique() :
    if df[df['channel_name'] == name].subscriberCount.unique()[0] > max_val :
        max_val = df[df['channel_name'] == name].subscriberCount.unique()[0]
        channel_max = name
    else :
        max_val = max_val
# print(channel_max, max_val)
print('La chaîne qui a le maximum d\'abonnés est : ', channel_max, ' et a : ', max_val, 'abonnés.') 

La chaîne qui a le maximum d'abonnés est :  Le Monde  et a :  568615 abonnés.


La valeur réelle trouvée sur Youtube est de 900 000 abonnés environ. Cela nous indique que les données ont dû être récoltés dans les 2 dernières années (estimation), et ne sont donc pas obsolètes. 


In [53]:
# moyenne et écart type du nombre moyen de mots utilisés en commentaires pour chaque chaîne.
#print(df.groupby('channel_name').nbrMotMoyenne.mean())
print('L\'écart type de nombre de mot moyen par commentaire par chaîne est de', '\n', df.groupby('channel_name').nbrMotMoyenne.mean().std())

L'écart type de nombre de mot moyen par commentaire chaîne est de 
 8.596640937578387
